In [ ]:
from flask import Flask, request, render_template, redirect, url_for, session, jsonify
import pickle
import os
import pathlib
import random
import string
from flask_mail import Mail, Message

app = Flask(__name__)

app.secret_key = 'secret_key'


# Email Configuration
app.config['MAIL_SERVER'] = 'smtp.gmail.com'
app.config['MAIL_PORT'] = 587
app.config['MAIL_USE_TLS'] = True
app.config['MAIL_USERNAME'] = 'kingkhandonkhan@gmail.com'  # Replace with your email
app.config['MAIL_PASSWORD'] = 'ghwalodrum'   # Replace with your email password
mail = Mail(app)

# Store generated OTPs temporarily (in a production app, use a database)
otp_storage = {}

# Account class definition
class Account:
    def __init__(self, accNo=0,email='', name='', deposit=0, type='', role='Customer'):
        self.accNo = accNo
        self.name = name
        self.deposit = deposit
        self.type = type
        self.email=email
        self.role=role

    def depositAmount(self, amount):
        self.deposit += amount

    def withdrawAmount(self, amount):
        if amount <= self.deposit:
            self.deposit -= amount
            return True
        return False

def writeAccountsFile(account):
    file = pathlib.Path("accounts.data")
    if file.exists():
        with open('accounts.data', 'rb') as infile:
            oldlist = pickle.load(infile)
        oldlist.append(account)
        os.remove('accounts.data')
    else:
        oldlist = [account]

    with open('newaccounts.data', 'wb') as outfile:
        pickle.dump(oldlist, outfile)
    os.rename('newaccounts.data', 'accounts.data')

# load all accounts
def loadAccounts():
    file = pathlib.Path("accounts.data")
    if file.exists():
        with open('accounts.data', 'rb') as infile:
            return pickle.load(infile)
    return []


def saveAccounts(accounts):
    with open('accounts.data', 'wb') as outfile:
        pickle.dump(accounts, outfile)

# Flask routes
@app.route('/')
def home():
    return render_template('home.html')


# Define the route to serve the account creation form
@app.route('/create_account_form', methods=['GET'])
def create_account_form():
    return render_template('create_account.html')

@app.route('/create_account', methods=['POST'])
def create_account():
    # Get role from the form
    role = request.form.get('role', 'Customer')  # Default role is 'Customer'
    
    # Common fields
    name = request.form.get('name', '')
    email = request.form.get('email', '')

    # Initialize Account object with defaults for optional fields
    account = Account(
        name=name,
        email=email,
        role=role
    )

    if role == 'Customer':
        # Process fields specific to customers
        account.accNo = int(request.form.get('accNo', 0))
        account.deposit = int(request.form.get('deposit', 0))
        account.type = request.form.get('type', '')

    # Write account to file or database
    writeAccountsFile(account)

    return redirect(url_for('home'))




# Helper functions
def loadAccounts():
    file = pathlib.Path("accounts.data")
    if file.exists():
        with open('accounts.data', 'rb') as infile:
            return pickle.load(infile)
    return []

# Route for login page
@app.route('/login_form', methods=['GET'])
def login_form():
    return render_template('login.html')

# Handle login
@app.route('/login', methods=['POST'])
def login():
    email = request.form['email']
    accounts = loadAccounts()

    # Check if the email exists in accounts
    for account in accounts:
        if account.email == email:
            # Generate a random 6-digit OTP
            otp = random.randint(100000, 999999)
            otp_storage[email] = otp

            # Send the OTP to the user's email
            msg = Message(
                subject="Your Login OTP",
                sender=app.config['MAIL_USERNAME'],
                recipients=[email]
            )
            msg.body = f"Your OTP for login is: {otp}. It is valid for 5 minutes."
            mail.send(msg)

            session['temp_email'] = email  # Store the email temporarily for verification
            return redirect(url_for('verify_otp'))

    return "Account not found! <a href='/login_form'>Try again</a>", 404

# OTP verification form
@app.route('/verify_otp', methods=['GET'])
def verify_otp():
    if 'temp_email' not in session:
        return redirect(url_for('login_form'))
    return render_template('verify_otp.html')

# Verify OTP
@app.route('/verify_otp', methods=['POST'])
def verify_otp_post():
    email = session.get('temp_email')
    if not email:
        return redirect(url_for('login_form'))

    entered_otp = int(request.form['otp'])
    if otp_storage.get(email) == entered_otp:
        accounts = loadAccounts()
        for account in accounts:
            if account.email == email:
                # Log the user in and store session data
                session['account'] = {
                    'accNo': account.accNo,
                    'name': account.name,
                    'email': account.email,
                    'type': account.type,
                    'deposit': account.deposit,
                    'role': account.role
                }
                otp_storage.pop(email, None)  # Remove OTP after successful login
                session.pop('temp_email', None)  # Clear temporary email
                return redirect(url_for('home'))

    return redirect(url_for('verify_otp'))

# Route to display account information
@app.route('/account_info', methods=['GET'])
def account_info():
    if 'account' in session:
        account = session['account']
        return render_template('account_info.html', **account)
    return redirect(url_for('login_form'))


@app.route('/all_accounts', methods=['GET'])
def all_accounts_page():
    return render_template('all_accounts.html')


# Deposit Form Route
@app.route('/deposit_form/<int:accNo>', methods=['GET'])
def deposit_form(accNo):
    return render_template('deposit_form.html', accNo=accNo)

# Withdraw Form Route
@app.route('/withdraw_form/<int:accNo>', methods=['GET'])
def withdraw_form(accNo):
    return render_template('withdraw_form.html', accNo=accNo)

# Delete Account Route
@app.route('/delete_account/<int:accNo>', methods=['GET'])
def delete_account(accNo):
    accounts = loadAccounts()
    accounts = [acc for acc in accounts if acc.accNo != accNo]
    saveAccounts(accounts)

    # Clear the session after account deletion
    session.pop('account', None)
    session.modified = True  # Mark session as modified to ensure it's saved
    

    return redirect(url_for('home'))



@app.route('/deposit/<int:accNo>', methods=['POST'])
def deposit(accNo):
    amount = request.form.get('amount', type=int)  # Get amount from form data
    accounts = loadAccounts()
    
    for account in accounts:
        if account.accNo == accNo:
            account.depositAmount(amount)  # Deposit the amount into the account
            saveAccounts(accounts)
            
            # Update session account with the new balance
            session['account']['deposit'] = account.deposit  # Update balance in session

            # Explicitly save the session to ensure changes are committed
            session.modified = True  # Mark session as modified to ensure it's saved

            return redirect(url_for('account_info'))

    return jsonify({'error': 'Account not found'}), 404



@app.route('/withdraw/<int:accNo>', methods=['POST'])
def withdraw(accNo):
    amount = request.form.get('amount', type=int)  # Get amount from form data
    accounts = loadAccounts()
    
    for account in accounts:
        if account.accNo == accNo:
            account.withdrawAmount(amount)  # Deposit the amount into the account
            saveAccounts(accounts)
            
            # Update session account with the new balance
            session['account']['deposit'] = account.deposit  # Update balance in session

            # Explicitly save the session to ensure changes are committed
            session.modified = True  # Mark session as modified to ensure it's saved

            return redirect(url_for('account_info'))

    return jsonify({'error': 'Account not found'}), 404

@app.route('/balance/<int:accNo>', methods=['POST'])
def balance(accNo):
    accounts = loadAccounts()
    for account in accounts:
        if account.accNo == accNo:
            return jsonify({'balance': account.deposit})
    return jsonify({'error': 'Account not found'}), 404

@app.route('/accounts', methods=['GET'])
def all_accounts():
    accounts = loadAccounts()
    return jsonify([{'accNo': acc.accNo, 'name': acc.name, 'type': acc.type, 'deposit': acc.deposit} for acc in accounts])

# @app.route('/delete_account/<int:accNo>', methods=['DELETE'])
# def delete_account(accNo):
#     accounts = loadAccounts()
#     accounts = [acc for acc in accounts if acc.accNo != accNo]
#     saveAccounts(accounts)
#     return jsonify({'message': 'Account deleted successfully'})

@app.route('/modify_account', methods=['PUT'])
def modify_account():
    data = request.json
    accounts = loadAccounts()
    for account in accounts:
        if account.accNo == data['accNo']:
            account.name = data.get('name', account.name)
            account.type = data.get('type', account.type)
            account.deposit = data.get('deposit', account.deposit)
            saveAccounts(accounts)
            return jsonify({'message': 'Account modified successfully'})
    return jsonify({'error': 'Account not found'}), 404

@app.route('/logout', methods=['GET'])
def logout():
    session.pop('account', None)
    return redirect(url_for('home'))


if __name__ == '__main__':
    app.run(host='127.0.0.1', port=5000, debug=False)


 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
